## Энергетический оракул
Ноутбук команды #12

Работа выполнена на основе модели LightGBM


### 1. Подготовка данных

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

import re

from tqdm import tqdm

from data_preprocess import DataTransformer
random_state = 12345
NUM_ITERATIONS = 5000

In [7]:
# Функция для вычисления метрики mae по дням из почасовых массивов данных

def mae_day(y_true, y_pred):
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    return mean_absolute_error(y_true_grouped, y_pred_grouped)
# Функция для вычисления метрик по дням из почасовых массивов данных

def metrics_day(y_true, y_pred):
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    mae = mean_absolute_error(y_true_grouped, y_pred_grouped)
    mape = mean_absolute_percentage_error(y_true_grouped, y_pred_grouped)
    r2 = r2_score(y_true_grouped, y_pred_grouped)
    return mae, mape, r2

#### 1.5 Чтение файлов с данными
Данные объединяются в один датасет

In [8]:
path = 'data'
transformer = DataTransformer() #инициализируем трансформер

In [9]:
all_ds, test_begin, test_end = transformer.open_file() #оставляем поле пустым что бы использовать открытый датасет

начало открытого теста: 2023-04-01 00:00:00     конец открытого теста: 2023-08-01 00:00:00


In [10]:
print(all_ds.shape)

(40152, 7)


In [11]:
all_ds = transformer.transform(all_ds)

date:  0
fill_weather_columns:  0
размер DS (40152, 13) дубликатов -  0
размер DS (43008, 15) дубликатов -  2856
размер DS (43008, 15) дубликатов -  2856
размер DS (43008, 15) дубликатов -  2856
holydays:  5712
create_lags:  168
VVP:  168
true weather:  168


In [ ]:
print(all_ds.shape)

In [ ]:
duplicates = all_ds.duplicated()

duplicates.sum()

#### 1.10 Демонстрация сформированного датасета

In [ ]:
# Итоговый набор колонок
all_ds.columns

In [ ]:
all_ds.head()

In [ ]:
all_ds[duplicates].sort_values(['date','time']).tail(56)

#### 1.11 Исключение лишних колонок

In [ ]:
# Отбираем признаки. Все лишние колонки здесь отбрасываем, кроме 'date', которую уберем позже 

feature_cols = list(all_ds.columns)

# выбрасываем взгляд в прошлое и расшифрованную погоду
drop_list = ['target', 'weather_pred', 'weather_fact', 'temp']

# выбрасываем признаки, найденные процедурно в процессе оптимизации
# КОМАНДЕ: здесь можно добавлять признаки на выброс с целью оптимизации
drop_list = drop_list + ['target_lag_48', 'target_lag_168', 'target_lag_336',
                        'target_lag_24', 'windy', 'clear',
                        'target_lag_72','has_rain_probability', #'temp_last_day',
                        'N', 'S', 'W', 'E', 'P','U', 'WW', 'Td', 'preholidays',  'cloudy',
 'rainy',
 'rain_probability','temp_pred', 'holidays', 'VVP'] 

for name in drop_list:
    feature_cols.remove(name)

# Итоговый список признаков
feature_cols

#### 1.12 Выделение наборов данных для обучения, валидации и тестирования

Выделялось два набора данных для обучения и валидации:
1. Обучение на данных с 2019 по 2021 с валидацией на 2022
2. Обучение на данных с 2019 по 2022 с валидацией на первом квартале 2023

Первый набор позволяет оценить влияние сезонности на обучение и предсказания, второй позволяет обучить модель на большем объеме данных и на более актуальных данных.

In [ ]:
# Формируем набор датасетов для обучения и проверки

features = all_ds[feature_cols]
target = all_ds['target']

# Функция для выделения временных интервалов из таблиц признаков и целей
# на этом этапе отбрасываем колонку 'date'
def features_interval(features, target, date1, date2):
    
    features_interval = features[ (features['date']>=date1) & (features['date']<date2) ]
    target_interval = target[features_interval.index]
    

    features_interval.loc[:, 'date'] = pd.to_datetime(features_interval['date'])

    # Преобразование столбца 'time' в timedelta
    features_interval.loc[:, 'time'] = pd.to_timedelta(features_interval['time'], unit='h')

    # Создание нового столбца 'datetime', объединяющего 'date' и 'time'
    features_interval.loc[:, 'datetime'] = features_interval['date'] + features_interval['time']

    # Установка столбца 'datetime' в качестве индекса
    features_interval.set_index('datetime', inplace=True)

    features_interval = features_interval.drop('date', axis=1)
    features_interval = features_interval.drop('time', axis=1)
    target_interval.index = features_interval.index

    return features_interval, target_interval


# для проверки на тестовой выборке будем учиться на всем тренировочном датасете
features_all_train, target_all_train = features_interval(features, target, '2019-01-01', test_begin)
features_open_test, target_open_test = features_interval(features, target, test_begin, test_end )


In [ ]:
features_all_train.head(5)

In [ ]:
duplicates_f = features_all_train.index.duplicated()

duplicates_f.sum()

In [ ]:
print(duplicates_f.shape)

In [ ]:
features_all_train[duplicates_f].sort_values('datetime').head(56)

In [ ]:
features_open_test.head(5)

## AUTO TS

In [ ]:
from autots import AutoTS, load_daily, load_hourly, load_live_daily, create_regressor
from autots.models.model_list import model_lists
import tensorflow as tf


In [ ]:
from pytorch_lightning.core.module import LightningModule


In [ ]:
import holidays

In [ ]:
russian = holidays.RU()

from sklearn.metrics import pairwise
#import neuralprophet
import scipy
from arch import arch_model
import pytorch_forecasting 
from neuralprophet import NeuralProphet
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler
import gluonts.model
from prophet import Prophet
import statsmodels.api as sm

In [ ]:
print(model_lists.keys())

In [ ]:
df_train_autots = pd.concat([target_all_train, features_all_train ], axis=1)

In [ ]:
df_train_autots.head()

In [ ]:
duplicates = df_train_autots.index.duplicated()

duplicates.sum()

In [ ]:
df_test_autots = pd.concat([target_open_test, features_open_test ], axis=1)
df_test_autots.head(3)

In [ ]:
#Config
forecast_length=24
frequency='infer'
drop_most_recent=0

In [ ]:
regr_train, regr_fcst = create_regressor(
    df_train_autots,
    forecast_length=forecast_length,
    frequency=frequency,
    drop_most_recent=drop_most_recent,
   # scale=True,
   # summarize="auto",
   # backfill="bfill",
   # fill_na="spline",
    #holiday_countries="RU",  # requires holidays package
    #encode_holiday_type=True,
    # datepart_method="simple_2",
)

In [ ]:
metric_weighting = {
    'mae_weighting': 5,
    'mape_weighting': 3,
    'rmse_weighting': 2,
#    'made_weighting': 0.5,
#    'mage_weighting': 1,
#    'mle_weighting': 0,
    'imle_weighting': 0,
#    'spl_weighting': 3,
    'containment_weighting': 0,
    'contour_weighting': 1,
    'runtime_weighting': 0.05,
}
model = AutoTS(
    forecast_length=forecast_length,
    frequency='infer',
    prediction_interval=0.95,
    ensemble='simple',
    model_list="regressor", #model_list_2,  "superfast", "default", "fast_parallel", 'fast'
    transformer_list="fast",  # "superfast",
    metric_weighting=metric_weighting,
    drop_most_recent=0,
    max_generations=1,
    n_jobs='auto',
    num_validations=0,
    validation_method="backwards",
    verbose=1,
    #holiday_country = "RU"
)

In [ ]:

model = model.import_template('AutoTS_best_models_n7.csv', method='only',  include_ensemble=True) # method='add on'
print("Overwrite template is: {}".format(str(model.initial_template)))

In [ ]:

model = model.fit(
    df_train_autots,
    weights={'target': 20}, future_regressor=regr_train
    
)

In [ ]:
import joblib
#joblib.dump(model, 'Autots_best_model_1.pkl')
#model = joblib.load('Autots_regr_temp_target.pkl')

In [ ]:
auto_ts_model = "AutoTS_best_models_n7.csv"  # .csv/.json
model.export_template(auto_ts_model, models='best',
                      n=7, max_per_model_class=3)

In [ ]:
prediction = model.predict()
# plot a sample
prediction.plot(model.df_wide_numeric,
                series=model.df_wide_numeric.columns[0],
                )

In [ ]:
print(prediction)

In [ ]:
prediction.forecast

In [ ]:
# accuracy of all tried model results
model_results = model.results()
# and aggregated from cross validation
validation_results = model.results("validation")

In [ ]:
validation_results.sort_values('mae', ascending=True).head(5) #.to_csv('result_val_auto_ts_2.csv')

In [ ]:
print(model)

In [ ]:
model.plot_validations()

In [ ]:
prediction = model.predict(forecast_length=24)
out = prediction.forecast
print(out)


In [ ]:
res_df = pd.DataFrame()
#res_df = pd.concat([res_df, out])
df_test_autots['date'] = df_test_autots.index.date
temp_train = df_train_autots
for date, df in df_test_autots.groupby('date'):
    temp_train_df = df.drop(columns='date')
    temp_train = pd.concat([temp_train, temp_train_df])
    
    
    try:
        model = model.fit(
            temp_train,
            
            weights={'target': 20} 
        )
        pred = model.predict()
        out = pred.forecast
    
        if out['target'].isnull().sum() != 0:
            print('Missing forecast ', date)
        
    except:
        print('Did not execute ', date)
    res_df = pd.concat([res_df, out])
df_test_autots.drop(columns='date')

In [ ]:
print(df_test_autots.shape, res_df.shape)

In [ ]:
mean_absolute_error(df_test_autots['target'], res_df['target'])

In [ ]:
        model = model.fit(
            temp_train,
            
            weights={'target': 20} 
        )
        pred = model.predict()
        out = pred.forecast

In [ ]:
df_test_autots.drop(columns='date')
df_test_autots.head()

In [ ]:
print(model.best_model_name)

In [ ]:
print(model.best_model_params)

In [ ]:

print(model.best_model_transformation_params)

### on new training
model = AutoTS(forecast_length=forecast_length,
               frequency='infer', max_generations=0,
               num_validations=0, verbose=0)
model = model.import_template(example_filename, method='only') # method='add on'
print("Overwrite template is: {}".format(str(model.initial_template)))


In [ ]:
# Рисуем графическое представление предсказания на 2022 год

y_true_copy = pd.DataFrame(target_valid).reset_index(drop=True)
y_true_copy['day'] = y_true_copy.index // 24
y_true_grouped = y_true_copy.groupby(by='day').sum()   
y_true_grouped
y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
y_pred_copy['day'] = y_pred_copy.index // 24
y_pred_grouped = y_pred_copy.groupby(by='day').sum()

#pd.DataFrame(date_valid)
tmpdf = pd.DataFrame(train_ds.loc[features_valid.index,:]['date']).groupby(by='date').count().reset_index().join(y_true_grouped)
tmpdf.plot(x='date', y='target', figsize=(18,5))
ax=plt.gca()
tmpdf = pd.DataFrame(train_ds.loc[features_valid.index,:]['date']).groupby(by='date').count().reset_index().join(y_pred_grouped)
tmpdf.plot(ax=ax, x='date', y=0)

In [ ]:
# Предсказываем той же моделью (19-21) тренировочный кусок 2023 (первый квартал)
mae = mae_day(target_2023, lgbm_model.predict(features_2023))
print(f'mae = {mae}')